In [38]:
import pandas as pd
import numpy as np
from nltk.tokenize import TweetTokenizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nifantova98_gmail_com/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [36]:
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

Installing mystem to /home/nifantova98_gmail_com/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [37]:
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    return token

In [25]:
df = pd.read_csv("ru-train.txt", sep="\t", header=None)


In [26]:
df.columnsumns = ["id", "s1", "s2", "score", "smth1", "smth2", "smth3"]

In [27]:
print(len(df))

22118837


In [28]:
df.head()

,id,s1,s2,score,smth1,smth2,smth3
0,ru-N0,"Я не сомневаюсь , что ты вернешься .","Я уверена , что ты вернешься .",73.6157,2.5,5,8
1,ru-N1,Это плод работы всей моей жизни .,Это труд всей моей жизни .,73.6157,2.5,5,6
2,ru-N2,Это плод не идёт ей на пользу .,Этот плод вредит Белле .,73.6157,2.5,5,11
3,ru-N3,"Эта история о том , как парень встретил девушку .",Это история о том как парень встретил девушку .,73.6157,2.5,5,3
4,ru-N4,"Что мы делаем , когда просыпаемся ?",Что мы делаем проснувшись ?,73.6157,2.5,5,12


In [30]:
tknzr = TweetTokenizer()

In [44]:
good_sentence_num = 0
y_theory = dict()
y_pred_base = dict()
y_pred_tuned = dict()
s1_embs_base = dict()
s1_embs_tuned = dict()
s2_embs_base = dict()
s2_embs_tuned = dict()
sim_min = 73.6157
sim_max = 73.6157
for index, row in df.iterrows():
    s1 = row["s1"]
    s1_words = preprocess_text(s1)
    s2 = row["s2"]
    s2_words = preprocess_text(s2)
    if row["sim"] < sim_min:
        sim_min = row["sim"]
    if row["sim"] > sim_max:
        sim_max = row["sim"]
    if len(s1_words) >= 5 and len(s2_words) >= 5:
        s1_emb_baseline = [0] * 300
        c1_baseline = 0
        s2_emb_baseline = [0] * 300
        c2_baseline = 0
        s1_emb_tuned = [0] * 300
        c1_tuned = 0
        s2_emb_tuned = [0] * 300
        c2_tuned = 0
        for w in s1_words:
            if w in russian_baselines.keys():
                s1_emb_baseline += russian_baselines[w]
                c1_baseline += 1
                s1_emb_tuned += russian_tuned[w]
                c1_tuned += 1
        for w in s2_words:
            if w in russian_baselines.keys():
                s2_emb_baseline += russian_baselines[w]
                c2_baseline += 1
                s2_emb_tuned += russian_tuned[w]
                c2_tuned += 1
        if c1_baseline >= 2 and c2_baseline >= 2 and c1_tuned >= 2 and c2_tuned >= 2:
            good_sentence_num += 1
            s1_emb_baseline = np.array(s1_emb_baseline) / c1_baseline
            s1_emb_tuned = np.array(s1_emb_tuned) / c1_tuned
            s2_emb_baseline = np.array(s2_emb_baseline) / c2_baseline
            s2_emb_tuned = np.array(s2_emb_tuned) / c2_tuned
            s1_embs_base[index] = s1_emb_baseline
            s1_embs_tuned[index] = s1_emb_tuned 
            s2_embs_base[index] = s2_emb_baseline
            s2_embs_tuned[index] = s2_emb_tuned 
            y_theory[index] = row["score"]
            cosine_base = cosine_similarity(np.array(s1_emb_baseline).reshape(1, -1), s2_emb_baseline.reshape(1, -1))
            cosine_tuned = cosine_similarity(s1_emb_tuned.reshape(1, -1), s2_emb_tuned.reshape(1, -1))
            y_pred_base[index] = cosine_base
            y_pred_tuned[index] = cosine_tuned

OSError: [Errno 12] Cannot allocate memory

In [32]:
print(good_sentence_num)

4378782
